1. Read the eos_map img 
2. Detect the player
3. Determine the player's direction
4. Calculate the rotating angle between shrine and the player

In [3]:
import cv2
import numpy as np
import math

# Input image from eos_map_x.png.
for num in range(3):
    print(num)
    img = cv2.imread(f'eos_map_{num}.png')
    # cv2.imshow('img', img)

    # detect the location of the player and shring
    mask1 = cv2.inRange(img, (200, 200, 0), (255, 255, 130)) # cyan color
    mask2 = cv2.inRange(img, (0, 0, 100), (50, 50, 255)) # red color
    player = cv2.bitwise_and(img, img, mask=mask1)
    shrine = cv2.bitwise_and(img, img, mask=mask2)

    # cv2.imshow('player', player)
    # cv2.imshow('shrine', shrine)

    # player to gray
    player_gray = cv2.cvtColor(player, cv2.COLOR_BGR2GRAY)

    # Calculate the center of player and the center will be the player_centre
    ret, thresh = cv2.threshold(player_gray, 125, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    player_moments = cv2.moments(thresh)
    cX = int(player_moments["m10"] / player_moments["m00"])
    cY = int(player_moments["m01"] / player_moments["m00"])
    player_centre = (cX, cY)

    # Create a contours
    img1 = np.zeros(img.shape, np.uint8) + 255
    img1 = cv2.drawContours(img1, contours, -1,(0,255,0),2)
    # Show in a window
    # cv2.imshow('Contours', img1)

    # make a triangle
    retval, triangle = cv2.minEnclosingTriangle(contours[0])
    triangle = np.int64(triangle)

    # Get the three points from triangle
    p1 = tuple(triangle[0][0])
    p2 = tuple(triangle[1][0])
    p3 = tuple(triangle[2][0])

    # Get the distance between the points and find the minimum
    dist_1 = math.dist(p1, p2)
    dist_2 = math.dist(p1, p3)
    dist_3 = math.dist(p2, p3)
    dist_min = min(dist_1, dist_2, dist_3)

    # From the minimum side knows the direction
    if (dist_min == dist_1):
        end_point = (2*p3[0]-player_centre[0], 2*p3[1]-player_centre[1])
    elif (dist_min == dist_2):
        end_point = (2*p1[0]-player_centre[0], 2*p1[1]-player_centre[1])
    else:
        end_point = (2*p2[0]-player_centre[0], 2*p2[1]-player_centre[1])

    # Yellow
    color = (0, 255, 255) 
    
    # Line thickness of 2 px 
    thickness = 2

    # Draw the player facing direction 
    output = img.copy()
    output = cv2.line(output, player_centre, end_point, color, thickness) 
    # cv2.imshow('Test',output)


    # Get the shring contours
    shrine_gray = cv2.cvtColor(shrine, cv2.COLOR_BGR2GRAY)

    contours, _ = cv2.findContours(shrine_gray.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_L1)

    # Because having multiple shrines so use the for loop to find out the minimum distance from shrine center to player center
    cur_dist = 1000000.0

    for i in range(len(contours)):
        moments = cv2.moments(contours[i])
        # print(moments)

        # There are some moments contain 0 or contour area smaller than 15 then we will skip them
        # print(cv2.contourArea(contours[i]))
        if(int(moments['m00']) == 0 or cv2.contourArea(contours[i]) < 15):
            continue
        cX = int(moments["m10"] / moments["m00"])
        cY = int(moments["m01"] / moments["m00"])
        # test = cv2.line(test, [cX, cY], player_centre, color, thickness) 

        # Distance to player centre
        dist_to_player = math.dist(player_centre, [cX, cY])
        # Find out the minimum distance
        cur_dist = min(dist_to_player, cur_dist)
        if(dist_to_player == cur_dist):
            shrine_centre = [cX, cY]

    # Draw a line from shrine centre to player centre
    output = cv2.line(output, shrine_centre, player_centre, color, thickness)
    # cv2.imshow(f'Output{num}',output)
    direction_angle = math.degrees(math.atan2(end_point[0]-player_centre[0], end_point[1]-player_centre[1]))
    shrine_angle = math.degrees(math.atan2(shrine_centre[0]-player_centre[0], shrine_centre[1]-player_centre[1]))
    angle = direction_angle - shrine_angle
    print(angle)
    if(angle > 0):
        rotation = "clockwise"
    elif(angle < 0):
        rotation = "counterclockwise"

    if(angle > 180 and angle > 0):
        angle = angle - 360
        rotation = "counterclockwise"
    elif(angle < -180 and angle < 0):
        angle = angle + 360
        rotation = "clockwise"
        
    font = cv2.FONT_HERSHEY_SIMPLEX 
    # org 
    org = (10, 20) 
    
    # fontScale 
    fontScale = 0.48
    
    # Blue color in BGR 
    color = (255, 255, 255) 
    
    # Line thickness of 2 px 
    # cv2.rectangle(output, (0, 0), (len(text)*8, 30), (0,0,0), -1)
    text = f"{rotation} {abs(round(angle,ndigits=2))} degree"
    thickness = 2
    output = cv2.putText(output, text, org, font,  
                   fontScale, color, thickness, cv2.LINE_4)
    cv2.imshow(f'output{num}',output)

cv2.waitKey(0)                                    # 按下任意鍵停止
cv2.destroyAllWindows()

0
-37.40535663140858
1
-216.4391104288354
2
68.96248897457818


這次利用以下幾個openCV函式完成此次作業:

- inrange()只將自己設定範圍的顏色作為mask，由於他的順序是gbr所以在調色上花了一些時間
- findContours()跟moment結合取得質心
- minEnclosingTriangle()中可以取得頂點以便這次去計算距離跟向量來找到玩家的方向
- 將算好的點用line()連線
- 在eos_map_2的部分shrine有被佔領的部分，但也被讀取為紅色，因此用contoursize去做篩選
- 度數的部分，當絕對值大於180時會將他轉成另一個方向(例如:順時鐘210度->逆時鐘150度)
- 最後有利用之前教的putText()把該旋轉的度數印上去

*程式上的細節都有在裡面註解